In [80]:
import numpy as np

In [81]:
class Operation(object):
    
    def __init__(self, input_nodes=[]):
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
            
    def compute(self):
        pass

In [82]:
class add(Operation):
    
    def __init__(self, x, y):
        super().__init__([x,y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var

In [83]:
class mul(Operation):
    
    def __init__(self, x, y):
        super().__init__([x,y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var * y_var

In [100]:
class dot(Operation):
    
    def __init__(self, x, y):
        super().__init__([x,y])
        
    def compute(self, x_mat, y_mat):
        self.inputs = [x_mat, y_mat]
        return x_mat.dot(y_mat)

In [101]:
class Placeholder(object):
    
    def __init__(self):
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [102]:
class Variables(object):
    
    def __init__(self, initial_value=None):
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

In [103]:
class Graph(object):
    
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        global _default_graph
        _default_graph = self

z = Ax + b

X = 10

b = 1

z = 10x + 1

In [104]:
g = Graph()

In [105]:
g.set_as_default()

In [106]:
A = Variables(10)
b = Variables(1)

In [107]:
x = Placeholder()
y = mul(A,x)

In [108]:
z = add(y,b)

In [109]:
# post order traversal of nodes
# just spits out the nodes in the correct order of operation, A*x but be before A*x + b
def traverse_postorder(operation):
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
        
    recurse(operation)
    return nodes_postorder

In [110]:
# Session is what executes the graphs
class Session(object):
    
    def run(self, operation, feed_dict={}):
        """
        feed_dict is the batches that we want to send into the graph
        """
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            if type(node) == Placeholder:
                node.output = feed_dict[node]
            elif type(node) == Variables:
                node.output = node.value
            else:
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)
                
            if type(node.output) == list:
                node.output = np.array(node.output)
                
        return operation.output

In [111]:
sess = Session()

In [112]:
results = sess.run(operation=z, feed_dict={x:10})

In [113]:
results

101

In [114]:
g = Graph()
g.set_as_default()

A = Variables([[10,20], [30,40]])
b = Variables([1,2])

x = Placeholder()

y = dot(A,x)
z = add(y,b)

In [115]:
sess = Session()

sess.run(operation=z, feed_dict={x:10})

array([[101, 202],
       [301, 402]])